# Axis Query Example

_Goal:_ demonstrate basic axis metadata handling using Pandas.

The CZ Cell Census stores obs (cell) metadata in a SOMA DataFrame, which can be queried and read as a Pandas DataFrame. The Cell Census also has a convenience package which simplifies opening the census.

Pandas DataFrame is an in-memory object. Take care that queries are small enough for results to fit in memory.

## Open the census

The `cell_census` Python package contains a convenient API to open the latest version of the Cell Census.

In [1]:
import cell_census

census = cell_census.open_soma()
human = census["census_data"]["homo_sapiens"]

## Summarize Census cell metadata

Tips:

- `read_as_pandas()` and `read_as_pandas_all()` return standard Pandas DataFrame objects, allowing the use of Pandas API.
- Queries will be much faster if you request only the DataFrame columns required for your analysis (e.g., `column_names=["cell_type_ontology_term_id"]`).
- You can also further refine query results by using a `value_filter`

### Example 1 - Summarize all cell types

This example reads the cell metadata (obs) into a Pandas DataFrame, and summarizes in a variety of ways using Pandas API.

In [2]:
# Read into a pandas dataframe.
obs_df = human.obs.read_as_pandas_all(column_names=["cell_type_ontology_term_id"])

# Use Pandas API to find all unique values in the `cell_type_ontology_term_id` column.
unique_cell_type_ontology_term_id = obs_df.cell_type_ontology_term_id.unique()

# Display only the first 10, as there are a LOT!
print(
    f"There are {len(unique_cell_type_ontology_term_id)} cell types in the Cell Census! The first 10 are:",
    unique_cell_type_ontology_term_id[0:10].tolist(),
)

# Using Pandas API, count the instances of each cell type term and return the top 10.
top_10 = obs_df.cell_type_ontology_term_id.value_counts()[0:10]
print("\nThe top 10 cell types and their counts are:")
print(top_10)

There are 545 cell types in the Cell Census! The first 10 are: ['CL:1000329', 'CL:0000787', 'CL:0000798', 'CL:0000909', 'CL:0000151', 'CL:1000348', 'CL:0000064', 'CL:0000576', 'CL:0000451', 'CL:0000898']

The top 10 cell types and their counts are:
CL:0000679    1889047
CL:0000235    1374219
CL:0000624    1286344
CL:0000860    1272977
CL:0000625    1244993
CL:0000623    1031420
CL:0000236     945552
CL:0001064     797557
CL:0000057     741330
CL:0000746     731139
Name: cell_type_ontology_term_id, dtype: int64


### Summarize a subset of cell types, selected with a `value_fitler`

This example utilizes a SOMA "value filter" to read the subset of cells with `tissue_ontologyy_term_id` equal to `UBERON:0002048` (lung tissue), and summarizes the query result using Pandas.

In [3]:
# Count cell_type occurrences for cells with tissue == 'lung'

# Read cell_type terms for cells which have a specific tissue term
LUNG_TISSUE = "UBERON:0002048"

obs_df = human.obs.read_as_pandas_all(
    column_names=["cell_type_ontology_term_id"],
    value_filter=f"tissue_ontology_term_id == '{LUNG_TISSUE}'",
)

# Use Pandas API to find all unique values in the `cell_type_ontology_term_id` column.
unique_cell_type_ontology_term_id = obs_df.cell_type_ontology_term_id.unique()

print(
    f"There are {len(unique_cell_type_ontology_term_id)} cell types in the Cell Census where tissue_ontology_term_id == {LUNG_TISSUE}! The first 10 are:",
    unique_cell_type_ontology_term_id[0:10].tolist(),
)

# Use Pandas API to count, and grab 10 most common
top_10 = obs_df.cell_type_ontology_term_id.value_counts()[0:10]
print(f"\nTop 10 cell types where tissue_ontology_term_id == {LUNG_TISSUE}")
print(top_10)

There are 176 cell types in the Cell Census where tissue_ontology_term_id == UBERON:0002048! The first 10 are: ['CL:0002325', 'CL:0000064', 'CL:0000875', 'CL:0000236', 'CL:0000623', 'CL:0000235', 'CL:0000084', 'CL:0000003', 'CL:0000186', 'CL:0000115']

Top 10 cell types where tissue_ontology_term_id == UBERON:0002048
CL:0000235    514828
CL:0000583    317503
CL:0000624    265512
CL:0000625    248053
CL:0000003    168203
CL:0000623    164002
CL:0000860    160365
CL:0001064    149067
CL:0002063    142612
CL:0002632    126058
Name: cell_type_ontology_term_id, dtype: int64


You can also define much more complex value filters. For example:
* combine terms with `and` and `or`
* use the `in` operator to query on multiple values

In [4]:
# You can also do more complex queries, such as testing for inclusion in a list of values

VENTRICLES = ["UBERON:0002082", "UBERON:OOO2084", "UBERON:0002080"]

obs_df = human.obs.read_as_pandas_all(
    column_names=["cell_type_ontology_term_id"],
    value_filter=f"tissue_ontology_term_id in {VENTRICLES}",
)

# Use Pandas API to summarize
top_10 = obs_df.cell_type_ontology_term_id.value_counts()[0:10]
display(top_10)

CL:0000746    158188
CL:0008034     84750
CL:0002548     79618
CL:0000115     64114
CL:0002131     61830
CL:0000763     31318
CL:0000669     27104
CL:0000003     22650
CL:0000057     19380
CL:0002144     18050
Name: cell_type_ontology_term_id, dtype: int64

## Full census stats

This example queries all organisms in the Census, and summarizes the diversity of various metadata lables.

In [5]:
COLS_TO_QUERY = [
    "cell_type_ontology_term_id",
    "assay_ontology_term_id",
    "tissue_ontology_term_id",
]

obs_df = {
    name: experiment.obs.read_as_pandas_all(column_names=COLS_TO_QUERY)
    for name, experiment in census["census_data"].items()
}

# Use Pandas API to summarize each organism
print(f"Complete cell census contains {sum(len(df) for df in obs_df.values())} cells.")
for organism, df in obs_df.items():
    print(organism)
    for col in COLS_TO_QUERY:
        print(f"\tUnique {col} values: {len(df[col].unique())}")

Complete cell census contains 37525571 cells.
homo_sapiens
	Unique cell_type_ontology_term_id values: 545
	Unique assay_ontology_term_id values: 21
	Unique tissue_ontology_term_id values: 178
mus_musculus
	Unique cell_type_ontology_term_id values: 206
	Unique assay_ontology_term_id values: 8
	Unique tissue_ontology_term_id values: 40
